## Week 12 - Class 11

This notebook have the most important exercises for theoretical class 11, which covers chapter 10 and 11 from the book. The labs from https://github.com/fchollet/deep-learning-with-python-notebooks are a guide for this notebook, mainly the labs 10,11



Dataset source:
https://keras.io/examples/timeseries/timeseries_weather_forecasting/

In [ ]:
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
#get the time series data, it is weather data
!wget https://s3.amazonaws.com/keras-datasets/jena_climate_2009_2016.csv.zip
!unzip jena_climate_2009_2016.csv.zip

In [ ]:
df=pd.read_csv('jena_climate_2009_2016.csv')

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
24*6*10
#every 10min

In [ ]:
from matplotlib import pyplot as plt
plt.plot(range(len(df['T (degC)'])), df['T (degC)'])

In [ ]:
#plot first 10 days

plt.plot(range(1440),  df['T (degC)'][:1440])

In [ ]:
df.columns

In [ ]:
#remove first column (we don't input timestamps)
df_raw=df[['p (mbar)', 'T (degC)', 'Tpot (K)', 'Tdew (degC)',
       'rh (%)', 'VPmax (mbar)', 'VPact (mbar)', 'VPdef (mbar)', 'sh (g/kg)',
       'H2OC (mmol/mol)', 'rho (g/m**3)', 'wv (m/s)', 'max. wv (m/s)',
       'wd (deg)']]

#or:
#df_raw = df.drop('Date Time', axis=1)

In [ ]:
raw_data=df_raw.to_numpy()

In [ ]:
raw_data.shape

In [ ]:
# define indexes of training, validation, testing datasets

num_train_samples = int(0.5 * len(raw_data))
num_val_samples = int(0.25 * len(raw_data))
num_test_samples = len(raw_data) - num_train_samples - num_val_samples
print("num_train_samples:", num_train_samples)
print("num_val_samples:", num_val_samples)
print("num_test_samples:", num_test_samples)

Since every feature has values with varying ranges, we do normalization to confine feature values to a range of [0, 1] before training a neural network. 

In [ ]:
#normalize the data

mean = raw_data[:num_train_samples].mean(axis=0)
raw_data -= mean
std = raw_data[:num_train_samples].std(axis=0)
raw_data /= std

Consider indices [0, 1, ... 99]. With sequence_length=10, sampling_rate=2, sequence_stride=3, shuffle=False


First sequence:  [0  2  4  6  8 10 12 14 16 18]

Second sequence: [3  5  7  9 11 13 15 17 19 21]

Third sequence:  [6  8 10 12 14 16 18 20 22 24]

...

Last sequence:   [78 80 82 84 86 88 90 92 94 96]

# timeseries_dataset_from_array

**sampling_rate** = Observations will be sampled at one data point per hour: we will only keep one data
point out of 6.


**sequence_length** = Observations will go back 5 days (120 hours). This is considering already the sampling rate.


**delay** = sampling_rate * (sequence_length + 24 - 1)—The target for a sequence will be the temperature 24 hours after the end of the sequence.

In [ ]:
sampling_rate = ...
sequence_length = ...
delay = ...
batch_size = 256

In [ ]:
train_dataset = keras.utils.timeseries_dataset_from_array(
    data=... ,
    targets=,
    sampling_rate=...,
    sequence_length=...,
    shuffle=True,
    batch_size=batch_size,
    start_index=...,
    end_index=...)

val_dataset = keras.utils.timeseries_dataset_from_array(
    data=..., #same
    targets=..., #same
    sampling_rate=..., #same
    sequence_length=...,#same
    shuffle=True,
    batch_size=batch_size,
    start_index=...,
    end_index=...)

test_dataset = keras.utils.timeseries_dataset_from_array(
    data=...,#same
    targets=...,#same
    sampling_rate=...,#same
    sequence_length=...,#same
    shuffle=True,
    batch_size=batch_size,
    start_index=...)

In [ ]:
for samples, targets in train_dataset:
    print("samples shape:", samples.shape)
    print("targets shape:", targets.shape)
    break

In [ ]:
inputs = keras.Input(shape=(sequence_length, raw_data.shape[-1]))
x = layers.LSTM(16)(inputs)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

callbacks = [
    keras.callbacks.ModelCheckpoint("jena_lstm.keras",
                                    save_best_only=True)
]
model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
history = model.fit(train_dataset,
                    epochs=2,
                    validation_data=val_dataset,
                    callbacks=callbacks)

model = keras.models.load_model("jena_lstm.keras")
print(f"Test MAE: {model.evaluate(test_dataset)[1]:.2f}")

In [ ]:
model.summary()

In [ ]:
import matplotlib.pyplot as plt
loss = history.history["mae"]
val_loss = history.history["val_mae"]
epochs = range(1, len(loss) + 1)
plt.figure()
plt.plot(epochs, loss, "bo", label="Training MAE")
plt.plot(epochs, val_loss, "b", label="Validation MAE")
plt.title("Training and validation MAE")
plt.legend()
plt.show()

LSTM explanation: https://colah.github.io/posts/2015-08-Understanding-LSTMs/

In [ ]:
# simple NN manual code
import numpy as np
timesteps = 100
input_features = 32
output_features = 64
inputs = np.random.random((timesteps, input_features))
state_t = np.zeros((output_features,))
W = np.random.random((output_features, input_features))
U = np.random.random((output_features, output_features))
b = np.random.random((output_features,))
successive_outputs = []
for input_t in inputs:
    output_t = np.tanh(np.dot(W, input_t) + np.dot(U, state_t) + b)
    successive_outputs.append(output_t)
    state_t = output_t
final_output_sequence = np.stack(successive_outputs, axis=0)

In [ ]:
#number of units in LSTM
#number of parmeters 
#latent dimension: because is the size of internal matrix
final_output_sequence.shape


In [ ]:
inputs.shape

In [ ]:
# simple NN, try different parameters for return_sequences

num_features = 14
steps = 120
inputs = keras.Input(shape=(steps, num_features))
outputs = layers.SimpleRNN(16, return_sequences=True)(inputs)
print(outputs.shape)

In [ ]:
# LSTM with dropout

inputs = keras.Input(shape=(sequence_length, raw_data.shape[-1]))
x = layers.LSTM(32, recurrent_dropout=0.25)(inputs)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

callbacks = [
    keras.callbacks.ModelCheckpoint("jena_lstm_dropout.keras",
                                    save_best_only=True)
]
model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
history = model.fit(train_dataset,
                    epochs=2,
                    validation_data=val_dataset,
                    callbacks=callbacks)


Same sequence but with text: https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/chapter11_part02_sequence-models.ipynb
